In [3]:
import ujson
import csv
import json_lines
import json
import sys
from elasticsearch import Elasticsearch
import matplotlib.pyplot as plt

csv.field_size_limit(sys.maxsize)

131072

In [9]:
ad_ids_remove = []
empty_clusters = []

with open("ad_ids_size_bias_removal.csv","r") as f:
    reader = csv.reader(f)
    for row in reader:
        ad_ids_remove.append(row[0])
        
        
with open("ad_ids_domain_bias_removal.csv","r") as f:
    reader = csv.reader(f)
    for row in reader:
        ad_ids_remove.append(row[0])

In [10]:
print len(ad_ids_remove)

110209


In [11]:
print len(set(ad_ids_remove))

110209


In [12]:
with open("empty_clusters.csv", "r") as f:
    reader = csv.reader(f)
    for row in reader:
        empty_clusters.append(row[0])
        
print len(empty_clusters)

103


In [11]:
new_train = []
cnt = 1

with open("train_adjusted.json", "r") as f:

    for row in json_lines.reader(f):
        if not row["_id"] in ad_ids_remove:
            new_train.append(row)
        cnt += 1

In [23]:
print len(new_train)

99334


In [26]:
# write out training data with docs removed
with open("training_removed.json", "w") as f:
    for doc in new_train:
        f.write(json.dumps(doc) + "\n")    

In [ ]:
# training data keys: [u'_type', u'_timestamp', u'_source', u'_score', 
#    u'_index', u'source', u'cluster_id', u'user', u'timestamp', u'_id', u'annotation']
# cdr keys: [u'_type', u'_timestamp', u'_source', u'_score', u'_index', u'_id']
# expanded list keys: ['cdr_id', 'phones', 'annotation', 'cluster_id', 'domain', 
#    'source', 'user', 'exp_cdr_ids', 'len_exp']

In [98]:
#https://memexproxy.com/wiki/pages/viewpage.action?pageId=8061410

ids_list = [] # to check for duplicates
duplicates = 0

docs_to_add = []
es = Elasticsearch("https://<user>:<password>@<es-instance>:9200/?pretty", verify_certs=True)

with open("expanded_positives_july20.csv", "r") as f:
    reader = csv.reader(f)
    for row in reader:
        print reader.line_num

        ids_list = eval(row[7])
        
        for _id in ids_list:

            query = {"query": {"term": { "_id": _id}}}
            cdr_docs = es.search(index = "memex-domains", doc_type = "escorts", body = query)

            for doc in cdr_docs["hits"]["hits"]:

                if not doc["_id"] in ids_list: # some ads are duplicated in cdr
                    ids_list.append(doc["_id"])

                    # Expand cdr doc to match training data
                    doc["source"] = row[5]
                    doc["cluster_id"] = row[3]
                    doc["user"] = row[6]
                    doc["annotation"] = row[2]

                    docs_to_add.append(doc) 
                else:
                    duplicates += 1

In [99]:
print duplicates
print len(docs_to_add)

854
11146


In [101]:
training_final = new_train + docs_to_add

In [102]:
with open("debiased_training.json", "w") as out:
    for row in training_final:
        out.write(json.dumps(row) + "\n")

In [4]:
with open("debiased_training.json", "r") as f:

    cluster_sizes = {}

    for ad in f.readlines():
        ad = ujson.loads(ad)
        if not ad["cluster_id"] in cluster_sizes:
            cluster_sizes[ad["cluster_id"]] = 1
        else:
            cluster_sizes[ad["cluster_id"]] += 1

  

In [12]:
print len(cluster_sizes.keys())
print json.dumps(cluster_sizes, indent=4)      

519
{
    "03f3465f2caada9d1a1764df15a13ef0d6515232": 272, 
    "2c73008838f3a50c4829e4444717bb6f22b6a142": 2, 
    "8ec8ffcf07d027fc57b68ea6dc656086d5f06c8b": 25, 
    "110812935b87cb0264ececdcb44b0faf9b784a33": 22, 
    "9ebbe2c32209607e70f0e1e54cf8e393c2f0e28c": 2054, 
    "14d64370eea446120699709c387d73e3269e0e43": 35, 
    "657dc39b6d631f93b8a2391b1bcfead01995a751": 25, 
    "73a4dbcbdc8eed793da6f11ae98a5bbb15b0b337": 31, 
    "50df923bfb03605799a023601f94d5b7bad62b32": 4, 
    "54af34cb0e007ef89934c2c4ec6c110bd3885571": 65, 
    "00224dc5dfba30436133d0ae68e48a4a69d00100": 55, 
    "fd9da2325a903775b3ba5738893c7da9ebd26ad7": 18, 
    "6d0659e1a9864d3b94136af468d3c96dcde44963": 51, 
    "c481c4d6721c08fd6024895c3d2a8fe71df745a9": 257, 
    "19ca42ec9862f6856d280d2b382f186478d427b4": 217, 
    "b6014ca6ae4cd4df39ce96e497e41df698715362": 15, 
    "ae5d544d1d9c8054d09bbe94c091c9f86c70df85": 146, 
    "77e64654c8ace63cbc81c2d3864c0fd736d32eda": 53, 
    "b8774f6c5d8e005738220e2cede7a03